https://adventofcode.com/2020/day/22

In [128]:
from collections import deque, Counter
import time

In [125]:
datafile = 'data/22-1.txt'

In [126]:
def load_data(txt):
    t1, t2 = txt.split('\n\n')
    d1 = [int(x) for x in t1.split('\n')[1:] if x.strip()]
    d2 = [int(x) for x in t2.split('\n')[1:] if x.strip()]
    return (d1, d2)

In [127]:
with open(datafile) as fh:
    data = load_data(fh.read())

In [4]:
testtxt = """\
Player 1:
9
2
6
3
1

Player 2:
5
8
4
7
10
"""

In [5]:
testdata = load_data(testtxt)

In [6]:
d1, d2 = [deque(x) for x in testdata]

In [7]:
d1

deque([9, 2, 6, 3, 1])

In [8]:
d2

deque([5, 8, 4, 7, 10])

In [9]:
def round(d1, d2):
    c1 = d1.popleft()
    c2 = d2.popleft()
    if c1 > c2:
        d1.append(c1)
        d1.append(c2)
    elif c2 > c1:
        d2.append(c2)
        d2.append(c1)
    else:
        raise ValueError("Can't happen: %s, %s", c1, c2)

In [10]:
round(d1, d2)
d1, d2

(deque([2, 6, 3, 1, 9, 5]), deque([8, 4, 7, 10]))

In [11]:
while d1 and d2:
    round(d1, d2)

In [12]:
d1, d2

(deque([]), deque([3, 2, 10, 6, 8, 5, 9, 4, 7, 1]))

In [13]:
def score(deck):
    return sum(i * c for (i, c) in enumerate(reversed(deck), 1))

In [14]:
score(d2)

306

In [15]:
with open(datafile) as fh:
    data = load_data(fh.read())

In [16]:
%%time
d1, d2 = [deque(x) for x in data]
i = 0
while d1 and d2:
    round(d1, d2)
    i += 1

CPU times: user 115 µs, sys: 7 µs, total: 122 µs
Wall time: 124 µs


In [17]:
i

395

In [18]:
len(d1)

50

In [19]:
len(d2)

0

In [20]:
score(d1)

33434

## Part 2

In [21]:
def trick(d1, d2):
    c1 = d1.popleft()
    c2 = d2.popleft()
    if c1 > c2:
        d1.append(c1)
        d1.append(c2)
    elif c2 > c1:
        d2.append(c2)
        d2.append(c1)
    else:
        raise ValueError("Can't happen: %s, %s", c1, c2)

In [113]:
d1 = deque([43, 19])
d2 = deque([2, 29, 44])

debug = 0

visited = set()
visited_hit_count = 0
for i in range(1_000):
    if debug:
        print(d1)
        print(d2)
        print()
    key = (tuple(d1), tuple(d2))
    if key in visited:
        visited_hit_count += 1
        if visited_hit_count == 2:
            print("2 visited hits")
            break
    visited.add(key)
    round(d1, d2)
    if not (d1 and d2):
        print("Winner")
        break
    if cyclecheck(d1, d2):
        print("cyclecheck")
        break
else:
    print("no end")
print(i)
print(d1)
print(d2)

Winner
3
deque([])
deque([19, 44, 43, 29, 2])


In [110]:
d1, d2

(deque([19, 43, 2]), deque([29, 30]))

In [102]:
d1 = deque([43, 19])
d2 = deque([2, 29, 44, 14, 16, 17, 18, 20])
list(d2)[:len(d1)]
d2[len(d1)]

44

In [111]:
def cyclecheck(d1, d2):
    len1, len2 = len(d1), len(d2)
    if len1 == len2:
        return False
    if len1 < len2:
        a, b = d1, d2
    else:
        a, b = d2, d1
    lena = len(a)
    if b[lena] > max(list(b)[:lena]):
        return False
    for i, (x, y) in enumerate(zip(a, b)):
        odd = i % 2
        if odd and x > y:
            return False
        elif not odd and x < y:
            return False
    return True

In [76]:
d1 = deque([43, 19, 21])
d2 = deque([2, 29, 14, 28])

print(d1)
print(d2)
print(cyclecheck(d1, d2))

deque([43, 19, 21])
deque([2, 29, 14, 28])
True


In [129]:
class CombatGame:
    
    def __init__(self, d1, d2, boredom_limit=10_000_000):
        self.d1 = deque(d1)
        self.d2 = deque(d2)
        self.boredom_limit = boredom_limit
        self.visited = set()
        
    def cyclecheck(self):
        key = (tuple(self.d1), tuple(self.d2))
        if key in self.visited:
            return True
        self.visited.add(key)
    
    def trick(self):
        if self.cyclecheck():
            return 'p1'
        d1, d2 = self.d1, self.d2
        c1, c2 = d1.popleft(), d2.popleft()
        if c1 <= len(d1) and c2 <= len(d2):
            self.subgame = CombatGame(list(d1)[:c1], list(d2)[:c2])
            t = time.time()
            print("recursion", t)
            winner = self.subgame.play()
            print("and we're back", t)
        else:
            if c1 > c2:
                winner = 'p1'
            elif c2 > c1:
                winner = 'p2'
            else:
                raise ValueError("can't happen")
        if winner == 'p1':
            d1.append(c1)
            d1.append(c2)
        else:
            d2.append(c2)
            d2.append(c1)
        if not (d1 and d2):
            return winner
    
    def play(self):
        for _ in range(self.boredom_limit):
            winner = self.trick()
            if winner:
                return winner
        else:
            raise BoredomError


def BoredomError(Exception):
    pass

In [120]:
d1 = [9, 2, 6, 3, 1]
d2 = [5, 8, 4, 7, 10]
game = CombatGame(d1, d2)
game.play()

'p2'

In [122]:
game.d1, game.d2

(deque([]), deque([7, 5, 6, 2, 4, 1, 10, 8, 9, 3]))

In [131]:
%%time
game = CombatGame(*data)
game.play()

recursion 1609630204.2136636
recursion 1609630204.2137878
and we're back 1609630204.2137878
and we're back 1609630204.2136636
recursion 1609630204.216611
recursion 1609630204.2166653
recursion 1609630204.2170177
recursion 1609630204.2170684
and we're back 1609630204.2170684
recursion 1609630204.217206
and we're back 1609630204.217206
and we're back 1609630204.2170177
recursion 1609630204.2174003
and we're back 1609630204.2174003
and we're back 1609630204.2166653
recursion 1609630204.2191687
and we're back 1609630204.2191687
recursion 1609630204.2198415
recursion 1609630204.2198853
and we're back 1609630204.2198853
recursion 1609630204.2202892
and we're back 1609630204.2202892
and we're back 1609630204.2198415
recursion 1609630204.2214656
and we're back 1609630204.2214656
recursion 1609630204.2215855
and we're back 1609630204.2215855
recursion 1609630204.221795
and we're back 1609630204.221795
recursion 1609630204.222635
recursion 1609630204.2227015
and we're back 1609630204.2227015
rec

and we're back 1609630204.4167717
recursion 1609630204.4179933
and we're back 1609630204.4179933
recursion 1609630204.4180803
recursion 1609630204.4181235
and we're back 1609630204.4181235
recursion 1609630204.4182405
recursion 1609630204.4183345
and we're back 1609630204.4183345
recursion 1609630204.4185855
and we're back 1609630204.4185855
and we're back 1609630204.4182405
recursion 1609630204.4188547
and we're back 1609630204.4188547
and we're back 1609630204.4180803
recursion 1609630204.4204545
and we're back 1609630204.4204545
recursion 1609630204.420535
and we're back 1609630204.420535
recursion 1609630204.4206178
recursion 1609630204.4206736
and we're back 1609630204.4206736
and we're back 1609630204.4206178
recursion 1609630204.4213252
recursion 1609630204.4213629
recursion 1609630204.421417
and we're back 1609630204.421417
and we're back 1609630204.4213629
recursion 1609630204.4216478
and we're back 1609630204.4216478
recursion 1609630204.4218152
and we're back 1609630204.4218

and we're back 1609630204.6177073
recursion 1609630204.6191394
recursion 1609630204.6191788
and we're back 1609630204.6191788
and we're back 1609630204.6191394
recursion 1609630204.6192966
and we're back 1609630204.6192966
recursion 1609630204.619361
and we're back 1609630204.619361
and we're back 1609630204.6123092
recursion 1609630204.619474
and we're back 1609630204.619474
recursion 1609630204.6196222
and we're back 1609630204.6196222
recursion 1609630204.6204581
and we're back 1609630204.6204581
recursion 1609630204.6205225
and we're back 1609630204.6205225
recursion 1609630204.6205854
and we're back 1609630204.6205854
recursion 1609630204.6208405
recursion 1609630204.6208858
and we're back 1609630204.6208858
recursion 1609630204.621011
and we're back 1609630204.621011
recursion 1609630204.6210754
and we're back 1609630204.6210754
recursion 1609630204.6212168
and we're back 1609630204.6212168
recursion 1609630204.621448
recursion 1609630204.62148
and we're back 1609630204.62148
and

and we're back 1609630204.8222463
recursion 1609630204.8228116
and we're back 1609630204.8228116
recursion 1609630204.8239613
and we're back 1609630204.8239613
and we're back 1609630204.816526
recursion 1609630204.8241467
and we're back 1609630204.8241467
recursion 1609630204.824935
recursion 1609630204.8249774
and we're back 1609630204.8249774
recursion 1609630204.825061
and we're back 1609630204.825061
recursion 1609630204.8260927
and we're back 1609630204.8260927
recursion 1609630204.8266401
and we're back 1609630204.8266401
recursion 1609630204.8267288
and we're back 1609630204.8267288
and we're back 1609630204.824935
recursion 1609630204.8269687
and we're back 1609630204.8269687
recursion 1609630204.827446
and we're back 1609630204.827446
recursion 1609630204.8283331
and we're back 1609630204.8283331
recursion 1609630204.828528
and we're back 1609630204.828528
recursion 1609630204.828673
recursion 1609630204.8287113
and we're back 1609630204.8287113
and we're back 1609630204.82867

and we're back 1609630205.028494
recursion 1609630205.0291545
and we're back 1609630205.0291545
and we're back 1609630205.0284634
recursion 1609630205.0298083
and we're back 1609630205.0298083
recursion 1609630205.030011
and we're back 1609630205.030011
and we're back 1609630205.0076122
recursion 1609630205.0304067
and we're back 1609630205.0304067
recursion 1609630205.030682
recursion 1609630205.0307813
and we're back 1609630205.0307813
recursion 1609630205.030866
and we're back 1609630205.030866
recursion 1609630205.0310385
and we're back 1609630205.0310385
recursion 1609630205.031101
and we're back 1609630205.031101
recursion 1609630205.0316384
and we're back 1609630205.0316384
recursion 1609630205.0319083
and we're back 1609630205.0319083
recursion 1609630205.0319893
recursion 1609630205.0320222
and we're back 1609630205.0320222
recursion 1609630205.0321386
and we're back 1609630205.0321386
and we're back 1609630205.0319893
recursion 1609630205.0322514
and we're back 1609630205.032

...

recursion 1609630215.171003
and we're back 1609630215.171003
recursion 1609630215.1714907
and we're back 1609630215.1714907
and we're back 1609630215.170968
recursion 1609630215.1715956
and we're back 1609630215.1715956
recursion 1609630215.1716762
and we're back 1609630215.1716762
recursion 1609630215.1719747
and we're back 1609630215.1719747
recursion 1609630215.1720397
and we're back 1609630215.1720397
recursion 1609630215.1721473
and we're back 1609630215.1721473
and we're back 1609630215.1708512
recursion 1609630215.172368
and we're back 1609630215.172368
recursion 1609630215.176069
and we're back 1609630215.176069
recursion 1609630215.1763008
and we're back 1609630215.1763008
recursion 1609630215.17708
recursion 1609630215.1772726
recursion 1609630215.177317
and we're back 1609630215.177317
recursion 1609630215.1773753
and we're back 1609630215.1773753
recursion 1609630215.1774533
and we're back 1609630215.1774533
and we're back 1609630215.1772726
recursion 1609630215.1776168
and

and we're back 1609630215.381367
recursion 1609630215.3829484
and we're back 1609630215.3829484
and we're back 1609630215.3806708
recursion 1609630215.3831384
and we're back 1609630215.3831384
recursion 1609630215.3840911
and we're back 1609630215.3840911
and we're back 1609630215.369856
recursion 1609630215.385048
and we're back 1609630215.385048
recursion 1609630215.4255416
and we're back 1609630215.4255416
recursion 1609630215.4257503
recursion 1609630215.4307864
and we're back 1609630215.4307864
and we're back 1609630215.4257503
recursion 1609630215.4368858
recursion 1609630215.436933
and we're back 1609630215.436933
recursion 1609630215.4370372
and we're back 1609630215.4370372
and we're back 1609630215.4368858
recursion 1609630215.4371915
recursion 1609630215.4372816
recursion 1609630215.4373424
and we're back 1609630215.4373424
recursion 1609630215.4374034
recursion 1609630215.43744
and we're back 1609630215.43744
and we're back 1609630215.4374034
recursion 1609630215.4375684
an

and we're back 1609630215.5808241
recursion 1609630215.5836854
recursion 1609630215.5837352
and we're back 1609630215.5837352
recursion 1609630215.5838451
and we're back 1609630215.5838451
recursion 1609630215.5839643
and we're back 1609630215.5839643
and we're back 1609630215.5836854
recursion 1609630215.584175
recursion 1609630215.5842073
and we're back 1609630215.5842073
recursion 1609630215.5855248
recursion 1609630215.585648
and we're back 1609630215.585648
recursion 1609630215.5857534
recursion 1609630215.5858455
and we're back 1609630215.5858455
and we're back 1609630215.5857534
recursion 1609630215.5897126
and we're back 1609630215.5897126
and we're back 1609630215.5855248
and we're back 1609630215.584175
recursion 1609630215.5903692
and we're back 1609630215.5903692
recursion 1609630215.591109
recursion 1609630215.5912042
and we're back 1609630215.5912042
recursion 1609630215.5913768
recursion 1609630215.5918138
and we're back 1609630215.5918138
and we're back 1609630215.59137

and we're back 1609630215.783964
recursion 1609630215.7855644
and we're back 1609630215.7855644
recursion 1609630215.7857084
recursion 1609630215.7857945
and we're back 1609630215.7857945
and we're back 1609630215.7857084
recursion 1609630215.7859266
and we're back 1609630215.7859266
recursion 1609630215.7860508
recursion 1609630215.786483
and we're back 1609630215.786483
and we're back 1609630215.7860508
recursion 1609630215.7879682
and we're back 1609630215.7879682
and we're back 1609630215.783926
recursion 1609630215.788204
recursion 1609630215.7882495
and we're back 1609630215.7882495
recursion 1609630215.7883062
and we're back 1609630215.7883062
recursion 1609630215.7885263
and we're back 1609630215.7885263
recursion 1609630215.7886147
and we're back 1609630215.7886147
recursion 1609630215.7888203
and we're back 1609630215.7888203
recursion 1609630215.792294
recursion 1609630215.792383
recursion 1609630215.7925706
and we're back 1609630215.7925706
and we're back 1609630215.792383


'p1'

In [132]:
game.d1, game.d2

(deque([34,
        17,
        7,
        2,
        13,
        1,
        48,
        32,
        22,
        6,
        27,
        4,
        24,
        11,
        47,
        36,
        29,
        23,
        14,
        12,
        43,
        42,
        41,
        31,
        37,
        16,
        26,
        19,
        50,
        33,
        39,
        28,
        46,
        35,
        49,
        18,
        45,
        5,
        44,
        30,
        40,
        9,
        38,
        21,
        25,
        8,
        20,
        15,
        10,
        3]),
 deque([]))

In [133]:
score(game.d1)

31657